In [1]:
import io
import re
import string
import sys
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    clean_text=' '.join([word for word in text.split() if word not in stopwords])
    return clean_text

stemmer = PorterStemmer()

data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')
data.drop(['ids', 'date', 'flag', 'user'],axis = 1, inplace = True)

#data cleaning
data['CleanText'] = data['text'].str.replace(r"http\S+","")

In [2]:
data['CleanText'] = data['CleanText'].str.replace("[^a-zA-Z]"," ")

In [3]:
data['CleanText'] = data['CleanText'].apply(lambda text : remove_stopwords(text.lower()))

In [4]:
data['CleanText'] = data['CleanText'].apply(lambda x: x.split())

In [6]:
data['CleanText'] = data['CleanText'].apply(lambda x:[stemmer.stem(i) for i in x])

In [7]:
data['CleanText'] = data['CleanText'].apply(lambda x: ' '.join([w for w in x]))

In [8]:
data.head()

,target,text,CleanText
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot awww bummer shoulda got david carr ...
1,0,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

cv= CountVectorizer()
X = cv.fit_transform(data['CleanText'])

In [2]:
import pickle

In [12]:
pickle.dump(cv,open('transform.pkl','wb'))

In [14]:
from sklearn.model_selection import train_test_split

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,data['target'] , test_size=.2,stratify=data['target'], random_state=42)

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [19]:
lr.predict(X_test)

array([4, 4, 4, ..., 0, 0, 4], dtype=int64)

In [20]:
pickle.dump(lr,open('model.pkl','wb'))

In [3]:
cvv = pickle.load(open('transform.pkl','rb'))

In [4]:
lrr = pickle.load(open('model.pkl','rb'))

In [5]:
s=['hello happy text']

In [6]:
vect = cvv.transform(s).toarray()

In [7]:
lrr.predict(vect)

array([4], dtype=int64)

In [10]:
print((lrr.predict(vect))[0])

4
